**Установка библиотек**

In [2]:


import sys
import subprocess
import pkg_resources

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("Установка необходимых библиотек... Это может занять 2–5 минут.\n")


install_package("torch")


install_package("torch-scatter -f https://data.pyg.org/whl/torch-${TORCH_VERSION}.html")
install_package("torch-sparse -f https://data.pyg.org/whl/torch-${TORCH_VERSION}.html")
install_package("torch-cluster -f https://data.pyg.org/whl/torch-${TORCH_VERSION}.html")
install_package("torch-spline-conv -f https://data.pyg.org/whl/torch-${TORCH_VERSION}.html")


install_package("torch-geometric")


install_package("networkx")
install_package("matplotlib")
install_package("scikit-learn")
install_package("numpy")

print("\nВсе библиотеки успешно установлены!")
print("Теперь можно запускать остальные блоки кода.")

Установка необходимых библиотек... Это может занять 2–5 минут.



CalledProcessError: Command '['C:\\Users\\Machcreator\\AppData\\Local\\Microsoft\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\python.exe', '-m', 'pip', 'install', 'torch-scatter -f https://data.pyg.org/whl/torch-${TORCH_VERSION}.html']' returned non-zero exit status 1.

**Импорт библиотек**

In [1]:

import torch
import torch.nn.functional as F
from torch_geometric.nn import GAE, VGAE, GCNConv
from torch_geometric.datasets import Planetoid, KarateClub
from torch_geometric.utils import train_test_split_edges
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score
import numpy as np

ModuleNotFoundError: No module named 'torch_geometric'

**Загрузка датасета**

In [ ]:

dataset = KarateClub()
data = dataset[0]
print(f"Граф: {data.num_nodes} узлов, {data.num_edges//2} рёбер")

**Разделение на train/test для unsupervised**

In [ ]:
data.train_mask = data.val_mask = data.test_mask = None
data = train_test_split_edges(data)

**Модель: Variational Graph Auto-Encoder (VGAE) с GCN encoder**

In [ ]:
class VGAEEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv_mu = GCNConv(2 * out_channels, out_channels)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

encoder = VGAEEncoder(dataset.num_features, 32)
model = VGAE(encoder)

**# Обучение (unsupervised reconstruction)**

In [ ]:

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(data.x, data.train_pos_edge_index)
    loss = model.recon_loss(z, data.train_pos_edge_index)
    loss += (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return loss.item()

for epoch in range(200):
    loss = train()
    if epoch % 50 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.4f}')

**Получение embeddings** 

In [ ]:
model.eval()
z = model.encode(data.x, data.train_pos_edge_index)
embeddings = z.detach().cpu().numpy()

**Clustering для сообществ (KMeans)**

In [ ]:

kmeans = KMeans(n_clusters=dataset.num_classes if hasattr(dataset, 'num_classes') else 4)  # Для Karate - 2-4
labels = kmeans.fit_predict(embeddings)

**Метрики (для Karate есть ground truth)**

In [ ]:
if 'karate' in str(dataset):
    ground_truth = nx.karate_club_graph().graph['labels']  # Стандартные 2 сообщества
    nmi = normalized_mutual_info_score(ground_truth, labels)
    print(f'NMI: {nmi:.4f}')

**Modularity (из networkx)**

In [ ]:

G = nx.Graph()
G.add_edges_from(data.edge_index.t().cpu().numpy())
modularity = nx.algorithms.community.modularity(G, [set(np.where(labels == i)[0]) for i in np.unique(labels)])
print(f'Modularity: {modularity:.4f}')


**Визуализация**

In [ ]:
pos = nx.spring_layout(G)
plt.figure(figsize=(10,8))
nx.draw(G, pos, node_color=labels, with_labels=True, cmap=plt.cm.tab10)
plt.title("Обнаруженные сообщества")
plt.show()